In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Novel Generation with LLM
* Langchain API를 이용해서 소설을 작성해볼 예정입니다.
* 직접 프롬프트를 제작하고, 이를 Langchain api에 입력해 소설을 출력해봅니다.

In [2]:
!pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [3]:
from typing import Dict, List

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key
* key 입력하기

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = # getpass.getpass()

In [5]:
TMP_PATH = "/content/drive/MyDrive/Lecture/LLMs/LLM & Langchain/Langchain examples/Projects/Novel_generation/single_prompt/prompt_template.txt" #set PATH

### Langchain prompt
* 유저 입력을 이용해서 prompt를 전달해봅시다.

In [6]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]] # 캐릭터 이름, 역할 (주연, 조연)
    idea: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def generate_novel(req: UserRequest) -> Dict[str, str]:
    writer_llm = ChatOpenAI(temperature=0.1, max_tokens=500, model="gpt-3.5-turbo")
    writer_prompt_template = ChatPromptTemplate.from_template(
        template=read_prompt_template(TMP_PATH)
    )
    writer_chain = LLMChain(
        llm=writer_llm, prompt=writer_prompt_template, output_key="output"
    )

    result = writer_chain(req.dict())

    return {"results": result["output"]}

### Prompt 참고자료 작성
* txt 파일에 존재하는 템플릿 프롬프트를 동작시킬 프롬프트를 작성합니다.
* user_data에 의해 작성된 데이터는 이미 생성된 템플릿 프롬프트에 추가되어 처리됩니다.

In [7]:
user_data = {
    "genre": "판타지",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "idea": "날씨가 추워지고 있습니다."
}


In [8]:
request_instance = UserRequest(**user_data)

In [9]:
generate_novel(request_instance)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'results': '[소설 제목: 얼음 마법사의 비밀]\n\n한 때 영웅으로 떠오른 김철수는 이제는 평범한 마을 주민으로 살아가고 있었다. 그러던 어느 날, 마을에 이상한 소문이 퍼지기 시작했다. 바로 날씨가 추워지고 있다는 것이었다. 이에 김철수는 마을을 구원하기 위해 힘을 모으기로 결심했다.\n\n그의 곁을 지키던 이영희는 김철수의 결심을 지지해주었다. 함께 날씨를 조절할 수 있는 마법사를 찾아 나선 둘은 얼음 마법사의 존재를 알게 된다. 그러나 얼음 마법사는 자신의 비밀을 지키려고 했고, 김철수와 이영희는 그의 비밀을 파헤치기 위한 모험을 떠나는데...\n\n얼음 마법사의 비밀을 풀기 위해 김철수와 이영희는 다양한 시험과 도전에 맞서야 했다. 그리고 그들은 마침내 얼음 마법사의 비밀을 밝혀내게 되는데, 그 비밀은 뜻밖의 충격과 감동을 안겨주었다. 함께 한 모험을 통해 김철수와 이영희는 더욱 깊은 우정을 쌓게 되었고, 마을을 구원하는 영웅으로 다시 떠올랐던 것이었다.\n\n얼음 마법사의 비밀'}